<a href="https://colab.research.google.com/github/pasquale90/pi-loop-console/blob/dev-v.0.2/d2m_groove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
gdrive='/content/gdrive'
drive.mount(gdrive, force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

#Download dataset

In [ ]:
'''
!pwd
%cd /content/gdrive/MyDrive
!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0.zip
#!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip
!unzip groove-v1.0.0.zip
!rm groove-v1.0.0.zip
!ls
datapath="/content/gdrive/MyDrive/groove"
'''

'\n!pwd\n%cd /content/gdrive/MyDrive\n!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0.zip\n#!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip\n!unzip groove-v1.0.0.zip\n!rm groove-v1.0.0.zip\n!ls\ndatapath="/content/gdrive/MyDrive/groove"\n'

In [ ]:
''' code provided by authors
import tensorflow as tf
import tensorflow_datasets as tfds

# tfds works in both Eager and Graph modes
tf.executing_eagerly()

# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset = tfds.load(
    name="groove/full-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True)

# Build your input pipeline
dataset = dataset.shuffle(1024).batch(32).prefetch(
    tf.data.experimental.AUTOTUNE)
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]
  print(f' Midi\n{midi}')
  print(f' Genre\n{genre}')
'''

In [ ]:
'''alternative dataset - midi only
!wget https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip
!unzip groove-v1.0.0-midionly.zip
!rm groove-v1.0.0-midionly.zip
!ls
'''

In [ ]:
#!cp -r /content/sample_data/groove /content/gdrive/MyDrive/

#Familiarization with mido

In [3]:
!pip install mido==1.2.9

     |████████████████████████████████| 52 kB 890 kB/s 


In [9]:
import os
import mido #import MidiFile, tempo2bpm, tick2second
import pandas as pd
import librosa

datapath="/data/pl/pi-loop-console/d2m/datasets/e-gmd/"

In [14]:
sample_midi_path=os.path.join(datapath,'drummer1/session1/1_funk_80_beat_4-4_36.midi')
mid = mido.MidiFile(sample_midi_path)#, clip=True
print(mid,"\n")

for i,track in enumerate(mid.tracks):
    print(f'track_{i}:\n {track}')

<midi file '/data/pl/pi-loop-console/d2m/datasets/e-gmd/drummer1/session1/1_funk_80_beat_4-4_36.midi' type 1, 2 tracks, 2412 messages> 

track_0:
 <midi track '' 4 messages>
track_1:
 <midi track '' 2408 messages>


# define classes

In [15]:
classes={"Bass":36,
        "Snare":38,
        "High_Tom":50,
        "Low_Mid_Tom":47,
        "High_Floor_Tom":43,
        "Open_Hi_Hat":46,
        "Closed_Hi_Hat":42,
        "Crash_Cymbal":49,
        "Ride_Cymbal":51}

def get_class_string(id):
  return [k for k,v in classes.items() if v == id][0]

print(get_class_string(36))

'''
classes=["Bass",
        "Snare",
        "High_Tom",
        "Low_Mid_Tom",
        "High_Floor_Tom",
        "Open_Hi_Hat",
        "Closed_Hi_Hat",
        "Crash_Cymbal",
        "Ride_Cymbal"]

def get_class_id(class_str):
  return [i for i,c in enumerate(classes) if c==class_str][0]

def get_class_given_id(id):
  return classes[id]

id=get_class_id("Ride_Cymbal")
print(id)
print(get_class_given_id(id))
'''
#print(classes)

Bass


'\nclasses=["Bass",\n        "Snare",\n        "High_Tom",\n        "Low_Mid_Tom",\n        "High_Floor_Tom",\n        "Open_Hi_Hat",\n        "Closed_Hi_Hat",\n        "Crash_Cymbal",\n        "Ride_Cymbal"]\n\ndef get_class_id(class_str):\n  return [i for i,c in enumerate(classes) if c==class_str][0]\n\ndef get_class_given_id(id):\n  return classes[id]\n\nid=get_class_id("Ride_Cymbal")\nprint(id)\nprint(get_class_given_id(id))\n'

# design Note and Midifile classes

In [16]:
'''
class Midifile:
  def __init__(self, filepath):
    self.midifile=mido.MidiFile(filepath)
    self.tempo=0
    self.bpm=0
    self.rythm_numerator=0
    self.rythm_denominator=0

    self.notes=[]

    self.PPQ=self.midifile.ticks_per_beat
    self.total_ticks=self.set_length_ticks()
    self.ending_tick=0

    #not explicitly required yet
    self.key=''
    self.clocks_per_click=0
    self.notated_32nd_notes_per_beat=0


  def get_midifile(self):
    return self.midifile

  def set_length_ticks(self):
    elapsed_ticks=0
    for track in self.midifile.tracks[0]:
      elapsed_ticks+=track.time
    self.total_ticks=elapsed_ticks

  def get_total_ticks(self):
    return self.total_ticks
  
  def get_total_seconds(self):
    return mido.tick2second(self.total_ticks,self.PPQ,self.tempo)

  def parse_midi_messages(self):

    for message in self.midifile.tracks[0]:
      message_type=message.type

      if message_type=="set_tempo":
        self.tempo=message.tempo
        self.bmp=int(mido.tempo2bpm(self.tempo))

      elif message_type=="time_signature":
        self.rythm_numerator=message.numerator
        self.rythm_denominator=message.denominator
        self.clocks_per_click=message.clocks_per_click
        self.notated_32nd_notes_per_beat=message.notated_32nd_notes_per_beat
        
      elif message_type=="key_signature":
        self.key=message.key
      
      elif message_type=="end_of_track":
        self.ending_tick=message.time
     
      else:
        #print(f"Unknown message {message}")
        pass
      
      #'' Initialize Notes and add them on the self.notes list - ignore program_change and control_change_messages
      elif message_type=="program_change":
        action="program_change??"
        channel=message.channel
        program=message.program 
        time=message.time
      
      elif message_type=="control_change":
        action="control_change??"
        channel=message.channel
        control=message.control
        value=message.value
        time=message.time
    
      elif message_type=="note_on":
        action="new_note"
        channel=message.channel 
        note=message.note
        note=remap_pitch(note)
        velocity=message.velocity
        time=message.time
      #''
      
  def get_ms_per_tick(self):
    return (60000 / (self.bpm * self.PPQ))
'''

'\nclass Midifile:\n  def __init__(self, filepath):\n    self.midifile=mido.MidiFile(filepath)\n    self.tempo=0\n    self.bpm=0\n    self.rythm_numerator=0\n    self.rythm_denominator=0\n\n    self.notes=[]\n\n    self.PPQ=self.midifile.ticks_per_beat\n    self.total_ticks=self.set_length_ticks()\n    self.ending_tick=0\n\n    #not explicitly required yet\n    self.key=\'\'\n    self.clocks_per_click=0\n    self.notated_32nd_notes_per_beat=0\n\n\n  def get_midifile(self):\n    return self.midifile\n\n  def set_length_ticks(self):\n    elapsed_ticks=0\n    for track in self.midifile.tracks[0]:\n      elapsed_ticks+=track.time\n    self.total_ticks=elapsed_ticks\n\n  def get_total_ticks(self):\n    return self.total_ticks\n  \n  def get_total_seconds(self):\n    return mido.tick2second(self.total_ticks,self.PPQ,self.tempo)\n\n  def parse_midi_messages(self):\n\n    for message in self.midifile.tracks[0]:\n      message_type=message.type\n\n      if message_type=="set_tempo":\n        se

In [17]:
#make a Note class
class Note:
  def __init__(self,channel,pitch,velocity,timestamp,timestamp_sec,duration,duration_sec):
    self.channel=channel
    self.pitch=self.remap_pitch(pitch)
    self.label=self.set_class(self.pitch)
    self.velocity=velocity
    self.timestamp=timestamp
    self.timestamp_sec=timestamp_sec
    self.duration=duration
    self.duration_sec=duration_sec
    
    self.relative_time=0 #in which part of the quarted is the note located

  def get_label(self):
    return self.label
  def get_pitch(self):
    return self.pitch
  def get_duration(self):
    return self.duration
  def get_velocity(self):
    return self.velocity
  def get_timestamp(self):
    return self.timestamp
  def get_timestamp_sec(self):
    return self.timestamp_sec
  def get_duration_sec(self):
    return self.duration_sec

  def remap_pitch(self,pitch):
    if pitch==36:
      return 36#"Bass"
    elif pitch in (37,38,40):
      return 38#"Snare"
    elif pitch in (48,50):
      return 50#"High_Tom"
    elif pitch in (45,47):
      return 47#"Low_Mid_Tom"
    elif pitch in (43,58):
      return 43#"High_Floor_Tom"
    elif pitch in (26,46):
      return 46#"Open_Hi_Hat"
    elif pitch in (22,42,44):
      return 42#"Closed_Hi_Hat"
    elif pitch in (49,52,55,57):
      return 49#"Crash_Cymbal"
    elif pitch in (51,53,59):
      return 51#"Ride_Cymbal"
      
  def set_class(self,id):
    classes={"Bass":36,
        "Snare":38,
        "High_Tom":50,
        "Low_Mid_Tom":47,
        "High_Floor_Tom":43,
        "Open_Hi_Hat":46,
        "Closed_Hi_Hat":42,
        "Crash_Cymbal":49,
        "Ride_Cymbal":51}
    return [k for k,v in classes.items() if v == id][0]
  '''
  def tick2second(tick, ticks_per_beat, tempo):
    """Convert absolute time in ticks to seconds.
    Returns absolute time in seconds for a chosen MIDI file time
    resolution (ticks per beat, also called PPQN or pulses per quarter
    note) and tempo (microseconds per beat).
    """
    scale = tempo * 1e-6 / ticks_per_beat
    return tick * scale
  '''

In [18]:
class Midifile:
  def __init__(self,filepath):
    self.midifile=mido.MidiFile(filepath)
    self.PPQ=self.midifile.ticks_per_beat
    self.notes=[]
    
    self.parse_midi_messages()
    

    #self.total_ticks=self.set_length_ticks()
    #self.ending_tick=0

   #not explicitly required yet
    #self.key=''
    #self.clocks_per_click=0
    #self.notated_32nd_notes_per_beat=0
  
  def parse_midi_messages(self):

    elapsed_ticks=0
    for m,message in enumerate(self.midifile.tracks[0]):
      message_type=message.type
      
      elapsed_ticks+=message.time
      print(f'message {m} , time= {message.time} elapsed_ticks={elapsed_ticks}')

      if message_type=="set_tempo":
        self.tempo=message.tempo
        self.bpm=int(mido.tempo2bpm(self.tempo))

      elif message_type=="time_signature":
        self.rythm_numerator=message.numerator
        self.rythm_denominator=message.denominator
        self.clocks_per_click=message.clocks_per_click
        self.notated_32nd_notes_per_beat=message.notated_32nd_notes_per_beat
        
      elif message_type=="key_signature":
        self.key=message.key
      
      elif message_type=="end_of_track":
        self.ending_tick=message.time
        self.total_ticks=elapsed_ticks

      elif message_type=="note_on":
        channel=message.channel 
        pitch=message.note
        velocity=message.velocity
        time=message.time
        #duration=self.midifile.tracks[0][m+1].time
        #print(f'duration={self.midifile.tracks[0][m+1].time}')
        timestamp=elapsed_ticks
        timestamp_sec=self.convert_tic2sec(timestamp)
        if velocity!=0:
          temp_counter=1
          duration=0
          for msg in self.midifile.tracks[0][m+1:]:
            duration+=msg.time
            if msg.type=="note_on" and msg.note==pitch and msg.velocity==0:
              new_note=Note(channel,pitch,velocity,timestamp,timestamp_sec,duration,self.convert_tic2sec(duration))
              self.notes.append(new_note)
              break
      else:
        #print(f"Unknown message {message}")
        pass

  def get_midifile(self):
    return self.midifile
  def get_tempo(self):
    return self.tempo
  def get_bpm(self):
    return self.bpm
  def get_rythm_numerator(self):
    return self.rythm_numerator
  def get_rythm_denominator(self):
    return self.rythm_denominator
  def get_note(self,idx):
    return self.notes[idx]
  def get_num_notes(self):
    return len(self.notes)
  def get_total_ticks(self):
    return self.total_ticks
  def get_total_seconds(self):
    return mido.tick2second(self.total_ticks,self.PPQ,self.tempo)
  def get_ms_per_tick(self):
    return (60000 / (self.bpm * self.PPQ))
  def convert_tic2sec(self,tick):
    return mido.tick2second(tick,self.PPQ,self.tempo)

In [26]:
#test Midifile class
test_midi=Midifile(sample_midi_path)
#test_midi.parse_midi_messages()
print(test_midi.get_tempo())
print(test_midi.get_bpm())

message 0 , time= 0 elapsed_ticks=0
message 1 , time= 0 elapsed_ticks=0
message 2 , time= 0 elapsed_ticks=0
message 3 , time= 1 elapsed_ticks=1
750000
80


In [25]:
#test Note class
num=test_midi.get_num_notes()
for i in range(num):
  n=test_midi.get_note(i)
  print(f'note {i+1}--> label:{n.get_label()},timestamp(ticks):{n.get_timestamp()}/{test_midi.get_total_ticks()}, \
  duration(ticks):{n.get_duration()}, \
  pitch:{n.get_pitch()}, \
  velocity:{n.get_velocity()} \
  timestamp(secs):{n.get_timestamp_sec()}/{test_midi.get_total_seconds()}, \
  duration(secs):{n.get_duration_sec()}')

# read midi files - csv

In [23]:
#read midi files using info.csv
csvpath=os.path.join(datapath,'e-gmd-v1.0.0.csv')
csvfile=pd.read_csv(csvpath,header=None)
print(csvfile)
print(type(csvfile))

             0                      1                        2             3   \
0       drummer                session                       id         style   
1      drummer1  drummer1/eval_session  drummer1/eval_session/1  funk/groove1   
2      drummer1  drummer1/eval_session  drummer1/eval_session/1  funk/groove1   
3      drummer1  drummer1/eval_session  drummer1/eval_session/1  funk/groove1   
4      drummer1  drummer1/eval_session  drummer1/eval_session/1  funk/groove1   
...         ...                    ...                      ...           ...   
45533  drummer9      drummer9/session1     drummer9/session1/12          rock   
45534  drummer9      drummer9/session1     drummer9/session1/12          rock   
45535  drummer9      drummer9/session1     drummer9/session1/12          rock   
45536  drummer9      drummer9/session1     drummer9/session1/12          rock   
45537  drummer9      drummer9/session1     drummer9/session1/12          rock   

        4          5       

In [24]:
midipaths=[]
#read midi files by iteratin through the folders
for folder in os.listdir(datapath):
  print (f'folder {folder}')
  folderpath=os.path.join(datapath,folder)
  if os.path.isdir(folderpath):
    for session_folder in os.listdir(folderpath):
      print(f'\tsession_folder {session_folder}')
      session_folderpath=os.path.join(folderpath,session_folder)
      if os.path.isdir(session_folderpath):
        for midifile in os.listdir(session_folderpath):
          #print('\t\t\t',midifile)
          midipath=os.path.join(midifile,session_folder)
          if midifile[-4:]=='.mid':
            midipaths.append(midipath)
print(f'{len(midipaths)} midifiles are found')

folder drummer1
	session_folder eval_session
	session_folder session1
	session_folder session2
	session_folder session3
folder drummer10
	session_folder session1
folder drummer3
	session_folder session1
	session_folder session2
folder drummer4
	session_folder session1
folder drummer5
	session_folder eval_session
	session_folder session1
	session_folder session2
folder drummer6
	session_folder session1
	session_folder session2
	session_folder session3
folder drummer7
	session_folder eval_session
	session_folder session1
	session_folder session2
	session_folder session3
folder drummer8
	session_folder eval_session
	session_folder session1
folder drummer9
	session_folder session1
folder e-gmd-v1.0.0.csv
folder LICENSE
folder README
0 midifiles are found


# list all files

In [17]:
def get_data_subpaths(datapath):
  folders=[]
  for c in os.listdir(datapath):
    subpath=os.path.join(datapath,c)
    if (os.path.isdir(subpath)):
      #print(f'{subpath} is a folder')
      folders.append(subpath)
    elif (c=="info.csv"):
      #print(f"{subpath} is the annotation file")
      csv_path=subpath
    else:
      #print(f'{c} is an irrelevant file - not of my concern!')
      pass
  return csv_path,folders

def read_csv(csv_path):
  midipaths,audiopaths,bpms,durations,splits,beat_types,sessions=[],[],[],[],[],[],[]
  csv=pd.read_csv(csv_path)  
  #print(csv.shape)
  idxs = csv.index.values.tolist()
  for i,idx in enumerate(idxs):
    #print(f'index {idx}, value {csv.loc[idx,:]}')
    midipaths.append(os.path.join(datapath,csv.loc[idx,"midi_filename"]))
    #audio_filename=csv.loc[idx,"audio_filename"])
    try:
      audiopaths.append(os.path.join(datapath,csv.loc[idx,"audio_filename"]))
    except TypeError:
      audiopaths.append("-")
    bpms.append(csv.loc[idx,"bpm"])
    durations.append(csv.loc[idx,"duration"])
    splits.append(csv.loc[idx,"split"])
    beat_types.append(csv.loc[idx,"beat_type"])
    sessions.append(csv.loc[idx,"session"])
  return midipaths,audiopaths,bpms,durations,splits,beat_types,sessions
  
csv_path,data_folders=get_data_subpaths(datapath)
midipaths,audiopaths,bpms,durations,splits,beat_types,sessions=read_csv(csv_path)
#for subf in data_folders:
#  print(os.listdir(subf))

In [18]:
print(len(midipaths))
print(len(audiopaths))
print(len(list(set(midipaths))))
print(len(list(set(audiopaths))))

1150
1150
1150
1091
